# StyleGAN2-ADA (PyTorch): training a model from scratch

Reference: https://github.com/woctezuma/steam-stylegan2-ada-pytorch

## Set-up the Colab machine

**Tip**: to ensure that your Colab session stays connected for a long duration, follow: https://stackoverflow.com/a/58275370/

### Check the GPU

In [ ]:
!nvidia-smi

### Install requirement

Reference: https://github.com/NVlabs/stylegan2-ada-pytorch/issues/2

In [ ]:
%pip install ninja

### Install my fork of StyleGAN2-ADA (PyTorch)

Reference: https://github.com/woctezuma/stylegan2-ada-pytorch/tree/google-colab

Switch to the branch `google-colab`.

In [ ]:
%cd /content/

%rm -rf stylegan2-ada-pytorch/
!git clone https://github.com/woctezuma/stylegan2-ada-pytorch.git

%cd stylegan2-ada-pytorch/
!git checkout google-colab

%cd /content/

### Download image data

Data consists of ~ 2500 Steam banners (256x256 resolution) in a 74 MB archive.

In [ ]:
from pathlib import Path

archive_name = 'steam-oneface-lr_with_retinaface.tar.gz'

if not Path(archive_name).exists():
  print('Downloading')
  !gdown --id 1-0Nk7H6Cn3Nt60EdHG_NWSA8ohi2oBqr
  !tar xf {archive_name}

### Prepare datasets

Reference: https://github.com/NVlabs/stylegan2-ada-pytorch#preparing-datasets

In [ ]:
custom_images = '/content/steam-oneface-lr/'
custom_dataset = '/content/datasets/steam/'

In [ ]:
!python stylegan2-ada-pytorch/dataset_tool.py \
 --source {custom_images} \
 --dest {custom_dataset}

## Mount Google Drive

In [ ]:
%pip install Google-Colab-Transfer

In [ ]:
import colab_transfer

colab_transfer.mount_google_drive()

In [ ]:
output_folder = '/content/drive/MyDrive/training-runs'

## Train new networks

Reference: https://github.com/NVlabs/stylegan2-ada-pytorch#training-new-networks

### Initial run (transfer learning)

Start with transfer learning from a model trained on a **diverse** dataset, e.g. `lsundog256`.

Caveat: the upstream model was trained with `cfg_map=8`.

In [ ]:
snap = 10
gamma = 10
kimg = 5000
mini_batch = 32

In [ ]:
!python stylegan2-ada-pytorch/train.py \
 --outdir={output_folder} \
 --snap={snap} \
 --metrics=none \
 --data={custom_dataset} \
 --cfg=auto_norp \
 --cifar_tune=1 \
 --gamma={gamma} \
 --kimg={kimg} \
 --batch={mini_batch} \
 --cfg_map=8 \
 --augpipe=bg \
 --freezed=10 \
 --resume=ffhq256


### Following runs (resume from the latest snapshot)

Then,  **automatically** resume from the latest pickle snapshot.

In [ ]:
!python stylegan2-ada-pytorch/train.py \
 --outdir={output_folder} \
 --snap={snap} \
 --metrics=none \
 --data={custom_dataset} \
 --cfg=auto_norp \
 --cifar_tune=1 \
 --gamma={gamma} \
 --kimg={kimg} \
 --batch={mini_batch} \
 --cfg_map=8 \
 --augpipe=bg \
 --freezed=10 \
 --resume=latest
